In [1]:
import sys
sys.path.append('C:\\Users\\temex\\Desktop\\mmdet3dProj') 
from models.adversarial.voxel_perturber import VoxelPerturber
import torch
import numpy as np

In [21]:
# Create mock data: 
# KITTI point clouds usually have the format [N, 4] where N is the number of points
# and the 4 channels typically represent [x, y, z, intensity].
# Note: Adjust 'num_points' to match typical KITTI sizes, say 10000 for a dense cloud.
num_points = 10000
mock_data = torch.randn(1, num_points, 4)  # Batch size is 1, for [batch, N, channels]

# Instantiate your VoxelPerturber
voxel_perturber = VoxelPerturber()

# Apply the perturber to your mock data
perturbed_data = voxel_perturber(mock_data)


In [24]:
# Check shapes
print("Original shape:", mock_data.shape)
print("Perturbed shape:", perturbed_data.shape)

# Optionally, inspect some points
print("Original data sample:", mock_data[0, :5, :])  # First 5 points
print("Perturbed data sample:", perturbed_data[0, :5, :])  # Corresponding perturbed points


Original shape: torch.Size([1, 10000, 4])
Perturbed shape: torch.Size([1, 10000, 4])
Original data sample: tensor([[-0.1494,  0.7043,  0.1950,  1.3358],
        [-1.5862, -0.7082, -0.0312, -0.3766],
        [ 0.8353, -0.1492, -1.4712, -1.5065],
        [ 0.4119, -0.6192, -2.3897,  1.0991],
        [-1.0990,  0.7724,  0.6767,  0.7247]])
Perturbed data sample: tensor([[ 0.5404, -1.7099,  0.6704, -1.0932],
        [ 2.1591,  0.1801,  0.3437,  0.5619],
        [-0.4073, -0.5649,  1.8453,  1.7852],
        [ 0.5790,  0.0590,  1.8478, -0.9582],
        [ 1.6981, -1.3159, -0.3688, -0.5466]], grad_fn=<SliceBackward0>)


In [25]:
file_path = 'C:\\Users\\temex\\Desktop\\mmdet3dProj\\mmdetection3d\\data\\kitti\\testing\\velodyne\\000000.bin'

# Load and format the point cloud data
point_cloud = np.fromfile(file_path, dtype=np.float32).reshape(-1, 4)
point_cloud_tensor = torch.tensor(point_cloud).unsqueeze(0)  # Add batch dimension

# Instantiate and apply your VoxelPerturber
voxel_perturber = VoxelPerturber()
perturbed_data = voxel_perturber(point_cloud_tensor)

# Output the perturbed data
print("Perturbed data shape:", perturbed_data.shape)
print("Original data shape:", point_cloud_tensor.shape)

Perturbed data shape: torch.Size([1, 125635, 4])
Original data shape: torch.Size([1, 125635, 4])


In [26]:
from models.detectors.adversarial_voxelnet import AdversarialVoxelNet
from mmdet3d.structures import CameraInstance3DBoxes, Box3DMode

# Mock data setup
num_points = 10000  # Number of points in point cloud
batch_size = 1      # Typically testing one example at a time
mock_points = torch.rand((batch_size, num_points, 4))  # Simulate [x, y, z, intensity]
mock_img_metas = [{'img_shape': (375, 1242), 'lidar2img': torch.randn(4, 4)}]
mock_gt_bboxes_3d = CameraInstance3DBoxes(torch.rand((10, 7)), box_dim=7, origin=(0.5, 0.5, 0.5))

# Mock model setup - replace with your actual config
adversary_config = {'type': 'SomeAdversary', 'perturb_magnitude': 0.1}  # Replace with actual adversary config
model = AdversarialVoxelNet(adversary=adversary_config)

# Forward pass
points = [mock_points]
img_metas = [mock_img_metas]
gt_bboxes_3d = [mock_gt_bboxes_3d.to('cpu')]
gt_labels_3d = [torch.randint(0, 3, (10,))]  # Example ground truth labels
outputs = model.forward_train(points, img_metas, gt_bboxes_3d, gt_labels_3d)

# Assertions - Adjust according to expected model behavior and output
assert 'loss_cls' in outputs, "Classification loss missing in outputs"
assert 'loss_bbox' in outputs, "BBox loss missing in outputs"
assert 'perturbation_norm' in outputs, "Perturbation norm missing in outputs"


TypeError: __init__() missing 3 required positional arguments: 'voxel_encoder', 'middle_encoder', and 'backbone'